In [5]:
from pedalboard import Pedalboard, Reverb, load_plugin, VST3Plugin
from pedalboard.io import AudioFile
from mido import Message # not part of Pedalboard, but convenient!
from utils import *

In [2]:
instrument = load_plugin("C:\\Program Files\\Common Files\\vst3\\Serum2.vst3\\Contents\\x86_64-win\\Serum2.vst3", plugin_name="Serum 2")

In [28]:
def filter_cat_params(params):
    cont_params = {}
    cat_params = {}
    for k, v in params.items():
        if len(v.valid_values) < 100:
            if 'semitones' in k or 'rate' in k or v.type == float:
                if v.range == (0.0, 1.0, 1.0):
                    cat_params[k] = v
                else:
                    cont_params[k] = v
            else:
                cat_params[k] = v
        else:
            cont_params[k] = v
    return cat_params, cont_params

In [29]:
target_params = filter_osc_params(instrument)
cat_params, cont_params = filter_cat_params(target_params)

In [30]:
print(len(target_params))
print(len(cat_params))
print(len(cont_params))

227
47
180


In [20]:
def get_value_range(param):
    valid_values = param.valid_values
    valid_raw_values = []
    for v in valid_values:
        if param.type == float:
            valid_raw_values.append(param.get_raw_value_for(v))
        elif param.type == str:
            valid_raw_values.append(param.get_raw_value_for(str(v)))
        elif param.type == bool:
            valid_raw_values.append(param.get_raw_value_for(v))
    min_raw, max_raw = min(valid_raw_values), max(valid_raw_values)
    step = (max_raw - min_raw) / (len(valid_values) - 1)
    return min_raw, max_raw, step

In [21]:
ranges = {}
cont_types = {}
for k, v in cont_params.items():
    if None in v.range:
        ranges[k] = get_value_range(v)
        cont_types[k] = v.type
    else:
        ranges[k] = v.range
        cont_types[k] = v.type
categories = {}
cat_types = {}
num_categories = {}
for k, v in cat_params.items():
    categories[k] = v.valid_values
    num_categories[k] = len(v.valid_values)
    cat_types[k] = v.type

In [32]:
def generate_random_param_vector(ranges, num_categories):
    param_vector = {}
    for k, v in ranges.items():
        param_vector[k] = random.uniform(v[0], v[1])
    for k, v in num_categories.items():
        param_vector[k] = random.randint(0, v - 1)
    return param_vector
generate_random_param_vector(ranges, num_categories)

{'a_level': 0.2411328034471859,
 'a_pan': 0.39332889591570575,
 'a_octave_oct': 0.9470365025480945,
 'a_semi_semitones': 11.447060671754365,
 'a_fine_cents': 51.87102199300284,
 'a_ratio': 0.33575651921698046,
 'a_hz_offset_hz': -243.3966032135611,
 'a_coarse_pitch': 0.28416553208328355,
 'a_start': 25.280326273432564,
 'a_end': 20.337559680608443,
 'a_scan_rate': 119.47686893137563,
 'a_scan_bpm_rate': 0.022423038571651732,
 'a_position': 35.03259764400776,
 'a_loop_start': 0.598193160243754,
 'a_loop_end': 59.79158766106202,
 'a_loop_x_fade': 32.07355236490621,
 'a_single_slice': 9.98533838483774,
 'a_unison': 2.556636274142349,
 'a_uni_detune': 0.6294985793549361,
 'a_uni_blend': 30.661427388521712,
 'a_uni_width': 27.777307629265692,
 'a_uni_span': -12.104188306977548,
 'a_uni_rand_start': 24.20314378507822,
 'a_uni_warp': -3.9827683439610553,
 'a_uni_warp_2': -26.337793125968247,
 'a_warp_var': 0.7285038201796027,
 'a_warp_2_var': 0.9447704455442297,
 'a_wt_pos': 121.9553060535859

In [34]:
config = {
    "ranges": ranges,
    "num_categories": num_categories,
    "cont_types": str(cont_types),
    "cat_types": str(cat_types)
}
import json
conf_file = "config.json"
with open(conf_file, "w") as f:
    json.dump(config, f)
